In [1]:
from create_data import create_csv

norm_sim = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/MCNorm/2016MD/halfSampleFeb22/job246-CombDVntuple-MCNorm-15144059-S28r2Restrip-firstHalf-2016MD-pKmumu-PF__PreselectedV1.root"
norm_real = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/realDataNorm/2016MD/halfSampleFeb22/job228-CombDVntuple-collision-firstHalf-2016MD-pKmumu-PF__PreselectedV1.root"

rare_sim = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/MC/2016MD/fullSampleOct2021/job207-CombDVntuple-15314000-MC2016MD_Full-pKmue-MC.root"
rare_real = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/realData/2016MD/halfSampleOct2021/blindedTriggeredL1520Selec-collision-firstHalf2016MD-pKmue_Fullv9.root"

In [3]:
kwargs = {
    'ver': '8.0.5',
    'sim_path': norm_sim,
    'actual_path': norm_real,
    'decay_tree_name': ':DTT1520mm/DecayTree',
    'preselect': False,
    'preselect_path': 'preselection.txt',
    'equalise_event_numbers': True,
    'dropnan': False,
    'restrict_mass': ([4500, 5200], [5800, 6500]),
    'train_fraction': 0.6,
    'val_fraction': 0.2,
    'test_fraction': 0.2,
    'isNormalisation': True,
    'DileptonQ2': True,
    'request': 'request.txt',
    'random_seed': 0
}

create_csv(kwargs)

/disk/homedisk/home/user211/project/create_data.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  user_features = user_features.append(add_row, ignore_index=True)


INFO: Reading in data from simulated tree
INFO: Simulated data manipulation complete
INFO: Restricting mass sidebands
INFO: Currently there are 45375 events
INFO: There are 27 events less than the lower mass threshold
INFO: There are 840 events above the higher mass threshold
INFO: There are 18538 events between threshold values
INFO: Mass restriction complete
INFO: Currently there are 25970 events
INFO: Sample currently includes 25970 background and 38057 signal events
INFO: Sample now includes 25970 background and 25970 signal events


/home/user211/envs/scientific/lib64/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/user211/envs/scientific/lib64/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Output to CSV: data_files/8.0.5/all.csv
Generating metadata file...
INFO: Metadata file generated at outputs/csv_metadata/8.0.5.txt
